## Azure 인증 설정

In [ ]:
import os
from azure.identity import AzureCliCredential

# ============================================
# 🔐 인증 방식 선택 (둘 중 하나만 True로 설정!)
# ============================================
USE_API_KEY = True           # API Key 기반 인증
USE_CLI_CREDENTIAL = False   # AzureCliCredential 사용 (az login 필요)

# ============================================

if USE_API_KEY and USE_CLI_CREDENTIAL:
    raise ValueError("❌ 둘 중 하나만 True로 설정하세요!")

if not USE_API_KEY and not USE_CLI_CREDENTIAL:
    raise ValueError("❌ 최소 하나는 True로 설정하세요!")

if USE_API_KEY:
    # API Key 인증 - 아래 값을 본인 정보로 수정!
    AZURE_OPENAI_API_KEY = "os.environ.get("AZURE_OPENAI_API_KEY", "").strip()"  # 👈 API Key 입력
    
    credential = None  # API Key 사용 시 credential 불필요
    os.environ["AZURE_OPENAI_API_KEY"] = AZURE_OPENAI_API_KEY
    
    print("🔐 API Key 인증 사용")
    print("   (API Key로 Azure OpenAI에 직접 연결)")
    
    if AZURE_OPENAI_API_KEY == "your-api-key-here":
        print("\n⚠️ API Key를 실제 값으로 수정하세요!")
    else:
        print("\n✅ API Key 설정 완료!")

elif USE_CLI_CREDENTIAL:
    credential = AzureCliCredential()
    print("🔐 AzureCliCredential 사용")
    print("   (터미널에서 'az login' 필요)")
    
    # 인증 테스트
    try:
        token = credential.get_token("https://cognitiveservices.azure.com/.default")
        print("\n✅ Azure CLI 인증 성공!")
    except Exception as e:
        print(f"\n❌ 인증 실패: {e}")
        print("   터미널에서 'az login'을 실행하세요.")

## Azure OpenAI 설정

In [ ]:
import os

#  아래 값을 본인의 Azure OpenAI 정보로 수정하세요!
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://hbjoo-mlg6s9pj-eastus2.cognitiveservices.azure.com"
os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"] = "gpt-4o"  # 또는 gpt-4.1

print("✅ Azure OpenAI 설정 완료!")
print(f"   Endpoint: {os.environ['AZURE_OPENAI_ENDPOINT']}")
print(f"   Deployment: {os.environ['AZURE_OPENAI_CHAT_DEPLOYMENT_NAME']}")

## STEP 1️⃣ DSR 기반 최대 대출 계산

In [ ]:
# 시중 은행 주택담보대출 평균 금리 조회 (한국은행 ECOS API)
# DATA/mortgage_rate.csv에 해당 월 금리가 있으면 사용, 없을 때만 API 호출 후 CSV에 저장
# 환경변수 BOK_ECOS_API_KEY 설정 시 API 사용, 미설정 시 기본값 4.48% 사용
import os
import csv
from datetime import date, datetime
from pathlib import Path
from dotenv import load_dotenv

# .env를 cwd부터 상위 폴더로 찾아서 로드 (노트북 cwd가 프로젝트 루트가 아닐 수 있음)
_cwd = Path.cwd().resolve()
for _ in range(6):
    _env = _cwd / ".env"
    if _env.exists():
        load_dotenv(_env)
        break
    if _cwd.parent == _cwd:
        break
    _cwd = _cwd.parent
else:
    load_dotenv()
print("BOK_ECOS_API_KEY:", "설정됨" if os.environ.get("BOK_ECOS_API_KEY", "").strip() else "미설정 (프로젝트 루트 .env 확인)")

_DEFAULT_RATE = 0.0448  # API 미사용 시 적용 (연 4.48%)

def _csv_path():
    return Path("DATA") / "mortgage_rate.csv"

def _minus_months(d, months):
    """날짜에서 months개월 뺀 날짜 반환 (월 단위)"""
    y, m = d.year, d.month
    m -= months
    while m <= 0:
        m += 12
        y -= 1
    return date(y, m, 1)

def _save_rate_to_csv(csv_path, yyyymm, rate):
    """해당 월 금리를 CSV에 한 줄 추가"""
    try:
        csv_path = Path(csv_path).resolve()
        csv_path.parent.mkdir(parents=True, exist_ok=True)
        file_exists = csv_path.exists()
        need_header = not file_exists or csv_path.stat().st_size == 0
        with open(csv_path, "a", encoding="utf-8", newline="") as f:
            w = csv.DictWriter(f, fieldnames=["yyyymm", "rate"])
            if need_header:
                w.writeheader()
            w.writerow({"yyyymm": yyyymm, "rate": rate})
    except Exception:
        pass

def get_mortgage_rate(as_of_date=None):
    """
    해당 날짜 기준 예금은행 주택담보대출 금리(연%)를 소수로 반환.
    ECOS는 약 2달 지연 공표이므로, 요청일 기준 2달 전 월 데이터를 API로 조회함.
    """
    if as_of_date is None:
        as_of_date = date.today()
    if isinstance(as_of_date, datetime):
        as_of_date = as_of_date.date()
    yyyymm = as_of_date.strftime("%Y%m")
    effective_date = _minus_months(as_of_date, 2)
    effective_yyyymm = effective_date.strftime("%Y%m")
    csv_path = _csv_path().resolve()
    if csv_path.exists():
        try:
            with open(csv_path, "r", encoding="utf-8") as f:
                r = csv.DictReader(f)
                for row in r:
                    if row.get("yyyymm") == yyyymm and row.get("rate", "").strip():
                        return float(row["rate"].strip())
        except Exception:
            pass
    api_key = os.environ.get("BOK_ECOS_API_KEY", "").strip()
    rate = None
    if api_key:
        try:
            import urllib.request
            import json
            # 121Y006 + BECBLA0302: 예금은행 대출금리(신규취급액 기준) 주택담보대출, 현재일 기준 2달 전 월 조회
            url = f"https://ecos.bok.or.kr/api/StatisticSearch/{api_key}/json/kr/1/10/121Y006/M/{effective_yyyymm}/{effective_yyyymm}/BECBLA0302/"
            print("url: ",url)
            with urllib.request.urlopen(url, timeout=10) as res:
                data = json.loads(res.read().decode())
            stat = data.get("StatisticSearch", {})
            result = stat.get("RESULT", {})
            if isinstance(result, dict) and result.get("CODE") != "000":
                pass
            else:
                rows = stat.get("row") or []
                if isinstance(rows, dict):
                    rows = [rows]
                if rows:
                    val = rows[0].get("DATA_VALUE")
                    if val is not None and str(val).strip():
                        rate = float(str(val).replace(",", ".").strip()) / 100.0
        except Exception:
            pass
        if rate is None:
            print(f"[get_mortgage_rate] ECOS API 호출했으나 금리 추출 실패. 기본금리({_DEFAULT_RATE*100:.2f}%) 사용.")
    if rate is None:
        rate = _DEFAULT_RATE
    _save_rate_to_csv(csv_path, yyyymm, rate)
    return rate


# DSR = (연간 모든 대출 원리금 상환액) / 연소득
def loan_by_dsr(annual_income, as_of_date=None):
    """
    as_of_date: 금리 적용 기준일 (date/datetime). None이면 오늘 기준 시중 금리 사용.
    """
    dsr = 0.4
    max_annual = annual_income * dsr
    r = get_mortgage_rate(as_of_date)
    n = 30
    factor = ((1 + r) ** n - 1) / (r * (1 + r) ** n)
    return max_annual * factor

## STEP 2️⃣ 취득세 함수

In [ ]:
def acquisition_tax(price):
    if price <= 600_000_000:
        return price * 0.011   # 1.1%
    elif price <= 900_000_000:
        return price * 0.022   # 근사치
    else:
        return price * 0.033   # 3.3%

## STEP 3️⃣ 전체 최대 매수가격 계산 (반복 탐색)

In [ ]:
## 최대 금액 계산기 로직
def max_purchase_price(
    annual_income,
    ltv,
    cash,
    as_of_date=None
):
    """as_of_date: 금리 적용 기준일. None이면 오늘 기준 시중 주택담보대출 평균 금리."""
    loan_dsr_limit = loan_by_dsr(annual_income, as_of_date=as_of_date)
    
    price = 100_000_000  # 시작값 1억
    step = 10_000_000    # 1천만원 단위 탐색
    
    max_price = 100_000_000
    
    while price < 200_000_000_000:
        loan_ltv = price * ltv
        loan = min(loan_dsr_limit, loan_ltv)
        
        tax = acquisition_tax(price)
        
        if price + tax <= loan + cash:
            max_price = price
            price += step
        else:
            break

    print("대출 가능 금액: ", loan)
    
    return max_price

## 호출 테스트

In [ ]:
# as_of_date 생략 시 오늘 기준 시중 주택담보대출 평균 금리 사용 (BOK ECOS API 또는 기본 4.48%)
result = max_purchase_price(
    annual_income=160_000_000,
    ltv=0.7,
    cash=320_000_000,
    as_of_date=None  # 예: date(2024, 6, 1) 지정 시 해당 월 금리 적용
)
print(f"최대 매수 가능 금액: {result:,} 원")

## 에이전트 생성

In [ ]:
from agent_framework.azure import AzureOpenAIChatClient

# Budget Agent 생성
if USE_API_KEY:
    chat_client = AzureOpenAIChatClient(
        api_key=os.environ["AZURE_OPENAI_API_KEY"]
    )
else:
    chat_client = AzureOpenAIChatClient(credential=credential)

agent = chat_client.as_agent(
    instructions="""
    아파트를 구매하는 사람의 최대 대출 가능 금액을 계산하는 어시스턴트입니다.
    사용자에게 아래의 내용을 물어보고 메모리에 저장하세요.
    - 연소득(부부라면 합산 연소득)
    - 생애첫구매여부
    - 보유현금
    
    항상 친절하고 간결하게, 한국어로 응답합니다.
    """,
    name="BudgetAgent"
)

print("✅ Budget Agent 생성 완료!")

## 메모리 클래스 정의

In [ ]:
from agent_framework import  ContextProvider, Context, ChatMessage
from collections.abc import Sequence
from typing import Any

class LoanInfoMemory(ContextProvider):
    """사용자 자산, 대출, 첫구매 여부 정보를 기억하는 메모리"""
    
    def __init__(self):
        self.expense_info = {
            "annual_income": None,
            "ltv": None,
            "cash": None
        }
    
    def extract_info(self, text: str) -> None:
        """텍스트에서 선호도 추출 (직접 호출용)"""
        text = text.lower()
        
        # ltv
        if "생애최초" in text or "처음" in text:
            self.expense_info["ltv"] = 0.7
        elif "1 주택" in text or "1주택" in text:
            self.expense_info["ltv"] = 0.4
        
        # 응답에서 annual_income 추출

        # 스타일
        if "모험" in text or "액티비티" in text:
            self.expense_info["style"] = "모험/액티비티"
        elif "휴양" in text or "휴식" in text:
            self.expense_info["style"] = "휴양/휴식"
        elif "문화" in text or "역사" in text:
            self.expense_info["style"] = "문화/역사"
        
        # 관심사
        for interest, keywords in [("음식", ["음식", "맛집"]), 
                                   ("쇼핑", ["쇼핑"]), 
                                   ("자연", ["자연", "산", "바다"])]:
            if any(k in text for k in keywords) and interest not in self.expense_info["interests"]:
                self.expense_info["interests"].append(interest)
    
    async def invoking(self, messages=None, **kwargs) -> Context | None:
        """에이전트 호출 전: 메모리 제공"""
        if any(self.expense_info.values()):
            info = f"""
[사용자 선호도 정보]
- 예산: {self.expense_info['budget'] or '미정'}
- 여행 스타일: {self.expense_info['style'] or '미정'}
- 관심사: {', '.join(self.expense_info['interests']) or '미정'}

위 정보를 참고하여 개인화된 추천을 제공하세요.
"""
            return Context(instructions=info)
        return None
    
    async def invoked(
        self,
        request_messages: ChatMessage | Sequence[ChatMessage],
        response_messages: ChatMessage | Sequence[ChatMessage] | None = None,
        invoke_exception: Exception | None = None,
        **kwargs: Any,
    ) -> None:
        """에이전트 호출 후: 선호도 추출 (자동)"""
        # request_messages를 리스트로 변환
        if isinstance(request_messages, ChatMessage):
            messages_list = [request_messages]
        else:
            messages_list = list(request_messages)
        
        # user 메시지에서 텍스트 추출
        for msg in messages_list:
            if hasattr(msg, 'role') and msg.role.value == "user":
                text = ""
                if hasattr(msg, 'contents') and msg.contents:
                    for content in msg.contents:
                        if hasattr(content, 'text'):
                            text += content.text
                        elif isinstance(content, str):
                            text += content
                        elif isinstance(content, dict) and 'text' in content:
                            text += content['text']
                
                if text:
                    self.extract_expense_info(text)

print("✅ Travelexpense_infoMemory 정의 완료")

In [ ]:
thread = budget_agent.get_new_thread()
agent_result = await agent.run("내가 살 수 있는 금액대의 아파트가 궁금해", thread=thread)

print("🤖 Agent 응답:")
print(agent_result.text)